# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [ ]:
#PEFT = LORA
#MODEL = "google-bert/bert-base-uncased"
#EVAL APPROACH. = REGULAR MODEL TRAINING ANDEVAL ACCURACY = 66%, LORA = 81%
#THIS IS WILL 500 LENGTH. DATASET FOR TRAINA NDTEST EACH AND 1 EPOCH
#FINE-TUNING DATASET:stanfordnlp/imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:

# Install the required version of datasets in case you have an older version
# You will need to choose "Kernel > Restart Kernel" from the menu after executing this cell

!pip install -q "datasets==2.15.0"



In [1]:
# Import the datasets and transformers packages

from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("stanfordnlp/imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
#tokenizer.pad_token = tokenizer.eos_token
# Example sentences
texts = ["Hello, world!", "This is a test."]

# Encode with padding
encoded_inputs = tokenizer(texts, padding=True, return_tensors='pt')
#encoded_inputs = tokenizer(texts)

print(encoded_inputs)

{'input_ids': tensor([[ 101, 7592, 1010, 2088,  999,  102,    0],
        [ 101, 2023, 2003, 1037, 3231, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1]])}


In [3]:


def preprocess_function(examples):
    """Preprocess the imdb dataset by returning tokenized examples."""
    token_ids = tokenizer(examples["text"],padding ='max_length', truncation=True, return_tensors="pt")
    #token_ids = tokenizer(examples["text"])
    return token_ids


tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = ds[split].map(preprocess_function, batched=True)


# Show the first example of the tokenized training set
print(tokenized_ds["train"][0]["input_ids"])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

[101, 2045, 2003, 2053, 7189, 2012, 2035, 2090, 3481, 3771, 1998, 6337, 2099, 2021, 1996, 2755, 2008, 2119, 2024, 2610, 2186, 2055, 6355, 6997, 1012, 6337, 2099, 3504, 15594, 2100, 1010, 3481, 3771, 3504, 4438, 1012, 6337, 2099, 14811, 2024, 3243, 3722, 1012, 3481, 3771, 1005, 1055, 5436, 2024, 2521, 2062, 8552, 1012, 1012, 1012, 3481, 3771, 3504, 2062, 2066, 3539, 8343, 1010, 2065, 2057, 2031, 2000, 3962, 12319, 1012, 1012, 1012, 1996, 2364, 2839, 2003, 5410, 1998, 6881, 2080, 1010, 2021, 2031, 1000, 17936, 6767, 7054, 3401, 1000, 1012, 2111, 2066, 2000, 12826, 1010, 2000, 3648, 1010, 2000, 16157, 1012, 2129, 2055, 2074, 9107, 1029, 6057, 2518, 2205, 1010, 2111, 3015, 3481, 3771, 3504, 2137, 2021, 1010, 2006, 1996, 2060, 2192, 1010, 9177, 2027, 9544, 2137, 2186, 1006, 999, 999, 999, 1007, 1012, 2672, 2009, 1005, 1055, 1996, 2653, 1010, 2030, 1996, 4382, 1010, 2021, 1045, 2228, 2023, 2186, 2003, 2062, 2394, 2084, 2137, 1012, 2011, 1996, 2126, 1010, 1996, 5889, 2024, 2428, 2204, 1998, 6

In [4]:
# ## Load and set up the model
# 
# We will now load the model and freeze most of the parameters of the model: everything except the classification head.

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},  # For converting predictions to strings
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
)

# Freeze all the parameters of the base model

for param in model.base_model.parameters():
    param.requires_grad = False

model.classifier

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Linear(in_features=768, out_features=2, bias=True)

In [5]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [6]:
import numpy as np

#import torch
#print(torch.version.cuda)
#!pip install bitsandbytes-cuda117

from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.668068,0.664000


Checkpoint destination directory ./data/sentiment_analysis/checkpoint-32 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=32, training_loss=0.7070152759552002, metrics={'train_runtime': 39.9308, 'train_samples_per_second': 12.522, 'train_steps_per_second': 0.801, 'total_flos': 131555527680000.0, 'train_loss': 0.7070152759552002, 'epoch': 1.0})

In [7]:
# ## Evaluate the model
# 
# Evaluating the model is as simple as calling the evaluate method on the trainer object. This will run the model on the test set and compute the metrics we specified in the compute_metrics function.



# Show the performance of the model on the test set
# What do you think the evaluation accuracy will be?
trainer.evaluate()


{'eval_loss': 0.6680682301521301,
 'eval_accuracy': 0.664,
 'eval_runtime': 19.1152,
 'eval_samples_per_second': 26.157,
 'eval_steps_per_second': 1.674,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
#ImportPeft modelLora

from peft import LoraConfig
from peft import get_peft_model

config = LoraConfig(r=16, lora_alpha= 32, lora_dropout = 0.1, 
                   target_modules = ["query", "key",  "value", "dense" ], task_type = "SEQ_CLS")

In [9]:
#GetLora modelcombined with base model
lora_model = get_peft_model(model, config)

# how many trainable parameters 
lora_model.print_trainable_parameters()

trainable params: 2,681,860 || all params: 112,164,100 || trainable%: 2.3910145937960543


In [10]:
#Training
trainer = Trainer(
    model=lora_model,
    args=TrainingArguments(
        output_dir="./peft/data/sentiment_analysis",
        learning_rate=2e-3,
        # Reduce the batch size if you don't have enough memory
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=1,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.554426,0.818000


Checkpoint destination directory ./peft/data/sentiment_analysis/checkpoint-32 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=32, training_loss=0.556954026222229, metrics={'train_runtime': 70.3123, 'train_samples_per_second': 7.111, 'train_steps_per_second': 0.455, 'total_flos': 135672502272000.0, 'train_loss': 0.556954026222229, 'epoch': 1.0})

In [11]:
#Save the trained weights for LORA
lora_model.save_pretrained("bert-base-uncased-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [13]:
#Load weights from Peftmodel for Inference

from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("bert-base-uncased-lora")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
#Evaluate 
trainer.evaluate()

{'eval_loss': 0.5544263124465942,
 'eval_accuracy': 0.818,
 'eval_runtime': 21.8738,
 'eval_samples_per_second': 22.858,
 'eval_steps_per_second': 1.463,
 'epoch': 1.0}